# Ship main particulars

In [1]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [1]:
import pandas as pd
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)
pd.set_option('display.width', 10000)
import numpy as np
import yaml

from vessel_manoeuvring_models.visualization.plot import track_plot, plot
from phd.visualization.plot_ship import track_plots
from wPCC_pipeline.pipelines.preprocess.nodes import load, add_thrust
from sklearn.metrics import mean_squared_error
import sympy as sp
from functools import reduce
from operator import add
from vessel_manoeuvring_models.symbols import *
import vessel_manoeuvring_models.symbols as symbols
from IPython.display import display
from vessel_manoeuvring_models.prime_system import df_prime
from vessel_manoeuvring_models.substitute_dynamic_symbols import run, lambdify, get_function_subs
from vessel_manoeuvring_models.parameters import df_parameters
from phd.helpers import mean
from vessel_manoeuvring_models.angles import mean_angle
from vessel_manoeuvring_models.angles import smallest_signed_angle
import inspect
from phd.visualization.plot_prediction import plot_total_force, plot_force_components
from vessel_manoeuvring_models.models.modular_simulator import ModularVesselSimulator, function_eq, subs_simpler
from scipy.optimize import least_squares
from vessel_manoeuvring_models.models.diff_eq_to_matrix import DiffEqToMatrix
from vessel_manoeuvring_models.prime_system import PrimeSystem
import statsmodels.api as sm
import logging
log = logging.getLogger(__name__)
import paper
import matplotlib.pyplot as plt
import os
from vessel_manoeuvring_models.prime_system import standard_units
plt.style.use('bmh')

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:10                                                                                   │
│                                                                                                  │
│    7                                                                                             │
│    8 from vessel_manoeuvring_models.visualization.plot import track_plot, plot                   │
│    9 from phd.visualization.plot_ship import track_plots                                         │
│ ❱ 10 from wPCC_pipeline.pipelines.preprocess.nodes import load, add_thrust                       │
│   11 from sklearn.metrics import mean_squared_error                                              │
│   12 import sympy as sp                                                                          │
│   13 from functools import reduce                                                                │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ModuleNotFoundError: No module named 'wPCC_pipeline'

In [4]:
%reload_kedro
ships=['wPCC','7m']

ship_datas = {ship:catalog.load(f"{ship}.ship_data") for ship in ships}


[07/18/23 16:12:44] INFO     Resolved project path as: /home/maa/dev/PHD.                           ]8;id=36858;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=435646;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/ipython/__init__.py#134\134]8;;\
                             To set a different path, run '%reload_kedro <project_root>'                           

[07/18/23 16:12:54] INFO     Kedro project phd                                                      ]8;id=981327;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=307677;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/ipython/__init__.py#103\103]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=589616;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=811193;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/ipython/__init__.py#104\104]8;;\
                             'pipelines'                                                                           

                    INFO     Registered line magic 'run_viz'                                        ]8;id=541925;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=604425;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/ipython/__init__.py#110\110]8;;\

                    INFO     Loading data from 'wPCC.ship_data' (YAMLDataSet)...                ]8;id=295746;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=745484;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py#343\343]8;;\

                    INFO     Loading data from '7m.ship_data' (YAMLDataSet)...                  ]8;id=109993;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=759390;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py#343\343]8;;\

In [2]:
df_main_particulars = pd.DataFrame(ship_datas).sort_index()
df_main_particulars.rename(columns={'wPCC':'5m'},inplace=True)
df_main_particulars.index.name='Particular'
df_main_particulars=df_main_particulars.select_dtypes(exclude='object')
df_main_particulars.drop(index=['TWIN'], inplace=True)
df_main_particulars.loc['x_G','5m'] = df_main_particulars.loc['x_G','7m']*df_main_particulars.loc['scale_factor','7m']/df_main_particulars.loc['scale_factor','5m']


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 df_main_particulars = pd.DataFrame(ship_datas).sort_index()                                  │
│   2 df_main_particulars.rename(columns={'wPCC':'5m'},inplace=True)                               │
│   3 df_main_particulars.index.name='Particular'                                                  │
│   4 df_main_particulars=df_main_particulars.select_dtypes(exclude='object')                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'ship_datas' is not defined

In [73]:
symbols_dict = {key:symbol for key,symbol in symbols.__dict__.items() if isinstance(symbol, sp.Symbol)}

In [74]:
table = df_main_particulars.applymap(np.format_float_positional, precision=4, unique=True, fractional=False)
table.reset_index(inplace=True)
table['Unit'] = table['Particular'].apply(lambda x:f"{standard_units.get(x,'').replace('_',' ')}")
table['Particular'] = table['Particular'].apply(lambda x:f"${symbols_dict.get(x,x)}$")

In [80]:
file_path = paper.file_path_with_nb_ref("main_particulars.csv")
table = table[['Particular','Unit','5m','7m']].copy()
table.to_csv(file_path,index=False)

In [76]:
table

,Particular,Unit,5m,7m
0,$A_R$,area,0.04124,0.07778
1,$A_XV$,area,0.5938,1.120
2,$A_YV$,area,3.396,6.405
3,$B$,length,0.9466,1.3
4,$CB$,-,0.4503,0.4503
5,$D$,length,0.1214,0.1667
6,$I_z$,inertia moment,693.1,3430.
7,$L$,length,5.015,6.887
8,$T$,length,0.2063,0.2833
9,$b_R$,length,0.1743,0.2393
